In [2]:
import sqlite3 
mshp_db_path = r"C:\Users\lgarzia\Documents\GitHub\explainability\local_data\data\mshp.sqlite" 
conn = sqlite3.connect(mshp_db_path)

In [53]:
c = conn.cursor()
c.execute("DROP TABLE pre_feature_eng_crs_ii")

In [54]:
sql = '''	
    CREATE TABLE pre_feature_eng_crs_ii AS
    select DISTINCT dcr.acc_rpt_num, dcr.acc_uuid, 
	coalesce(dcr.Name, ii.Name) name,  
	coalesce(dcr."Personal Injury", ii."Injury Type") injury_type, 
	coalesce(dcr.Age, ii.Age) age,
	coalesce(dcr."Person City/State", ii."City/State") city_state,
	dcr."Safety Device" AS safety_device, 
    dcr.Date AS date, 
    dcr.Time AS time, 
    dcr.Troop AS troop, 
    ii.Gender AS gender,
	ii."Veh. #" as veh_num, 
	ii."Involvement" AS involvement
	from daily_crash_report dcr
	left outer join injury_information ii on dcr.acc_uuid = ii.acc_uuid  --13751, inner(10129)
	and dcr.Name = ii.Name 
	and dcr.Age = ii.Age 
	and dcr."Person City/State" = ii."City/State"
    where dcr.Name != 'JUVENILE,'
'''
# 	--and dcr."Personal Injury" = ii."Injury Type"

In [55]:
c = conn.cursor()
c.execute(sql)

In [56]:
import dataprep.connector as dbc
import urllib
table_name = "injury_information"
mshp_db_path = r"C:\Users\lgarzia\Documents\GitHub\explainability\local_data\data\mshp.sqlite" 
mshp_db_path = urllib.parse.quote(mshp_db_path)
mshp_db_conn = "sqlite://"+mshp_db_path

In [74]:
df = dbc.read_sql(mshp_db_conn, f"SELECT * FROM pre_feature_eng_crs_ii")

In [58]:
df.shape

(12127, 13)

In [75]:
df.groupby('involvement').count()
# big drop in occupants from raw injury table
# theory is Juvenile are overwhelming percentage of OCCUPANTS

,acc_rpt_num,acc_uuid,name,injury_type,age,city_state,safety_device,date,time,troop,gender,veh_num
involvement,,,,,,,,,,,,
BICYCLIST,14,14,14,14,14,14,14,14,14,14,14,14
DRIVER,6761,6761,6761,6761,6761,6761,6761,6761,6761,6761,6761,6761
OCCUPANT,1803,1803,1803,1803,1803,1803,1803,1803,1803,1803,1803,1803
OTHER,2,2,2,2,2,2,2,2,2,2,2,2
PEDESTRIAN,140,140,140,140,140,140,140,140,140,140,140,140


In [59]:
df.count()

acc_rpt_num      12127
acc_uuid         12127
name             12127
injury_type      12127
age              12127
city_state       12097
safety_device    12127
date             12127
time             12127
troop            12127
gender            8720
veh_num           8720
involvement       8720
dtype: int64

In [60]:
# question is it possible to back into missing gender
# idea - get most from vehicle

In [61]:
c = conn.cursor()
c.execute("DROP TABLE pre_feature_eng")

In [62]:
sql = ''' 
CREATE TABLE pre_feature_eng AS

select distinct cii.acc_rpt_num, cii.acc_uuid, cii.name, cii.injury_type, cii.age, 
cii.city_state, cii.safety_device, cii.date, cii.time, cii.troop, 
coalesce(cii.gender, vi."Driver Gender") AS gender, 
coalesce(cii.veh_num, vi."Veh. #") AS veh_num,
coalesce(cii.involvement, "DRIVER") AS involvement,
vi."Vehicle Direction" AS vehicle_direction

from pre_feature_eng_crs_ii cii
left outer join vehicle_information vi 
on  cii.acc_uuid = vi.acc_uuid  --13751, inner(10129)
and cii.name = vi."Driver Name" 
and cii.age = vi."Driver Age" 
and cii."city_state" = vi."Driver City/State"
'''

In [63]:
c = conn.cursor()
c.execute(sql)

In [64]:
df = dbc.read_sql(mshp_db_conn, f"SELECT * FROM pre_feature_eng") 

In [65]:
df.count() # 12130 versus 12127 picked up some dupes will ignore for now

acc_rpt_num          12130
acc_uuid             12130
name                 12130
injury_type          12130
age                  12130
city_state           12100
safety_device        12130
date                 12130
time                 12130
troop                12130
gender               12100
veh_num              12100
involvement          12130
vehicle_direction    10086
dtype: int64

In [76]:
sql = '''
CREATE TABLE pre_feature_eng_base AS
With veh_dir AS 
(
SELECT vi.acc_uuid, vi."Veh. #" AS veh_num, MAX(vi."Vehicle Direction") AS vehicle_direction
FROM vehicle_information vi
GROUP BY vi.acc_uuid, vi."Veh. #"

)
SELECT e.acc_rpt_num, e.acc_uuid, e.name, e.injury_type, e.age, 
e.city_state, e.safety_device, e.date, e.time, e.troop, 
e.gender, e.veh_num, e.involvement,
coalesce(e.vehicle_direction, d.vehicle_direction) AS vehicle_direction
FROM pre_feature_eng e 
LEFT join veh_dir d on e.acc_uuid = d.acc_uuid AND e.veh_num = d.veh_num
'''

In [77]:
c = conn.cursor()
c.execute(sql)

In [78]:
df = dbc.read_sql(mshp_db_conn,"SELECT * FROM pre_feature_eng_base") 

In [79]:
df.count()

acc_rpt_num          12130
acc_uuid             12130
name                 12130
injury_type          12130
age                  12130
city_state           12100
safety_device        12130
date                 12130
time                 12130
troop                12130
gender               12100
veh_num              12100
involvement          12130
vehicle_direction    12098
dtype: int64

In [29]:
from ydata_profiling import ProfileReport

Pandas backend loaded 1.5.3
Numpy backend loaded 1.23.5
Pyspark backend NOT loaded
Python backend loaded


In [80]:
table_name = "pre_feature_eng_base"
profile = ProfileReport(df, title=f"Profiling Report for {table_name}")

In [81]:
profile.to_file(f"Profiling Report for {table_name}")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\lgarzia\Documents\GitHub\explainability\venv\lib\site-packages\ydata_profiling\profile_report.py:355: UserWarning: Extension  not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
# Question - still missing gender - what of 55
    # ANS - fixed issue with join on Injury Type - no have dupes that need to be cleaned

# Question - 3 missing vehicle direction - can we obtain the remainder with a join on acc_uuid & veh_num
# ANS - yes
# Question - would removing Juveniles help
# ANS - yes -> proceed with removing for analysis Name != 'JUVENILE,'

In [36]:
import numpy as np
df[df.gender.isnull()].head(n=10)
# '831f4197-f4aa-4cfe-b45b-b5fb865a04f0' <-- involvement isn't right
# appears Injury Type in None versus NO INJURY 

,acc_rpt_num,acc_uuid,name,injury_type,age,city_state,safety_device,date,time,troop,gender,veh_num,involvement,vehicle_direction
143,220121269,831f4197-f4aa-4cfe-b45b-b5fb865a04f0,"MILBOURN, KAYTIE L",NO INJURY,20,"KIDDER, MO",UNKNOWN,03/10/2022,5:33PM,H,None,<NA>,DRIVER,None
144,220121269,3506e870-f374-4aa9-93fb-cd0e42dbabb0,"ORR, BRIDGETTE L",NO INJURY,22,"HAMILTON, MO",NO,03/10/2022,5:33PM,H,None,<NA>,DRIVER,None
628,220147421,e3b1166a-9226-4490-a23d-6dbafd4d8eb9,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,03/25/2022,8:05AM,C,None,<NA>,DRIVER,None
629,220147363,58235742-d6c7-414e-b151-c853f90922ca,"HESTER, STEPHEN L",NO INJURY,18,"MOUNTAIN VIEW, MO",YES,03/25/2022,7:30AM,G,None,<NA>,DRIVER,None
687,220154476,278223ac-acf2-49ef-a055-05056f55bb47,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,03/28/2022,3:00PM,D,None,<NA>,DRIVER,None
891,220164885,13725b6c-5692-4a0e-90b5-e841013fd5ed,"UNK, UNK",NO INJURY,UNK,None,UNKNOWN,04/03/2022,12:37PM,C,None,<NA>,DRIVER,None
906,220164473,8bee3e66-7112-40e7-a6ef-6a9958b732e7,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,04/03/2022,2:14AM,D,None,<NA>,DRIVER,None
1063,220178007,adaed995-a897-4af2-af55-b5e567845edb,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,04/09/2022,8:30PM,G,None,<NA>,DRIVER,None
1426,220199768,de62c379-2cec-42fe-a6ce-a826bbbc48ed,"JUVENILE,",NO INJURY,0,"LATHROP, MO",YES,04/21/2022,3:20PM,A,None,<NA>,DRIVER,None
1639,220209109,f25715fb-4098-4930-a7d6-4d49609c7660,"PAYNE, EVELYN N",NO INJURY,39,"MARSHALL, MO",YES,04/26/2022,11:59AM,A,None,<NA>,DRIVER,None
